In [1]:
import requests
import json
import pandas as pd

from urllib.request import urlopen

In [2]:
tich_toons = ['Doofydoodler', 'Brindar', 'Whizzletit', 'Whizzletoot', 'Mostpaster', 'Tauryy']
column_list = ['character', 'realm', 'date', 'time', 'dungeon', 'short_name', 'mythic_level', 'upgrade', 'score', 'clear_time', 'clear_time_ms', 'par_time_ms', 'affix_1', 'affix_2', 'affix_3', 'affix_4', 'url']
recent_keys_df = pd.DataFrame([], columns=column_list)
display(recent_keys_df)

,character,realm,date,time,dungeon,short_name,mythic_level,upgrade,score,clear_time,clear_time_ms,par_time_ms,affix_1,affix_2,affix_3,affix_4,url


In [3]:
#adds the character and realm columns to each entry of the data frame, also converts clear_time_ms to minutes and seconds.
def character_data(character_name, realm):
    if realm == 'Tich': realm = 'Tichondrius'
    global recent_keys_df
    url = f"https://raider.io/api/v1/characters/profile?region=us&realm={realm}&name={character_name}&fields=mythic_plus_recent_runs"
    r = requests.get(url)
    data = r.json()
    df = pd.json_normalize(data, record_path = ['mythic_plus_recent_runs'])
    df['date'] = df.completed_at.str[0:10]
    df['time'] = df['completed_at'].str[11:19]
    clear_time = []
    for ms in df['clear_time_ms']:
        ms = int(ms)
        seconds=(ms/1000)%60
        seconds = int(seconds)
        minutes=(ms/(1000*60))%60
        minutes = int(minutes)
        hours=(ms/(1000*60*60))%24
        output = ("%d:%d:%d" % (hours, minutes, seconds))
        #print(output)
        clear_time.append(output)
    #print(clear_time)
    df['clear_time'] = clear_time
    df['character'] = character_name
    df['realm'] = realm
    df = pd.concat([df.drop(['affixes'], axis=1), df['affixes'].apply(pd.Series)], axis=1)
    for i in range(4):
        df = pd.concat([df.drop([i], axis=1), df[i].apply(pd.Series)], axis=1)
        df = df.drop(['icon'], axis=1)
        df = df.drop(['wowhead_url'], axis=1)
        df = df.drop(['description'], axis=1)
        df = df.drop(['id'], axis=1)
        df = df.rename(columns={'name': 'affix_'+str(i+1)})
    df = df.drop(['completed_at'], axis=1)
    df = df.drop(['map_challenge_mode_id'], axis=1)
    df = df.drop(['zone_id'], axis=1)
    df = df.rename(columns={"num_keystone_upgrades": "upgrade"})
    recent_keys_df = recent_keys_df.append(df, ignore_index=True)
    recent_keys_df = recent_keys_df.drop_duplicates(keep='last')

In [4]:
for toon in tich_toons:
    character_data(toon, 'Tich')
character_data('Ater', 'Stormrage')
character_data('Tauryy', 'Shattered-Halls')
recent_keys_df.to_csv('allkeys.csv', index=False)
display(recent_keys_df)

,character,realm,date,time,dungeon,short_name,mythic_level,upgrade,score,clear_time,clear_time_ms,par_time_ms,affix_1,affix_2,affix_3,affix_4,url,0
0,Doofydoodler,Tichondrius,2022-10-22,08:02:59,Tazavesh: Streets of Wonder,STRT,20,2,153.4,0:28:26,1706030,2340999,Fortified,Spiteful,Necrotic,Shrouded,https://raider.io/mythic-plus-runs/season-sl-4...,NaN
1,Doofydoodler,Tichondrius,2022-10-11,04:57:49,Mechagon Workshop,WORK,15,2,128.5,0:23:10,1390616,1920999,Fortified,Raging,Explosive,Shrouded,https://raider.io/mythic-plus-runs/season-sl-4...,NaN
2,Doofydoodler,Tichondrius,2022-10-02,00:00:04,Mechagon Workshop,WORK,22,0,154.8,0:32:33,1953560,1920999,Tyrannical,Sanguine,Storming,Shrouded,https://raider.io/mythic-plus-runs/season-sl-4...,NaN
3,Doofydoodler,Tichondrius,2022-09-28,01:28:41,Mechagon Junkyard,YARD,22,2,163.8,0:26:20,1580923,2280999,Tyrannical,Sanguine,Storming,Shrouded,https://raider.io/mythic-plus-runs/season-sl-4...,NaN
4,Doofydoodler,Tichondrius,2022-09-27,08:45:53,Grimrail Depot,GD,16,2,134.9,0:18:9,1089102,1800999,Fortified,Bolstering,Quaking,Shrouded,https://raider.io/mythic-plus-runs/season-sl-4...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,Tauryy,Shattered-Halls,2022-10-13,03:47:53,Iron Docks,ID,17,1,136.6,0:26:15,1575768,1800999,Tyrannical,Bursting,Volcanic,Shrouded,https://raider.io/mythic-plus-runs/season-sl-4...,NaN
75,Tauryy,Shattered-Halls,2022-10-10,09:58:46,Grimrail Depot,GD,23,2,168.9,0:20:42,1242372,1800999,Fortified,Raging,Explosive,Shrouded,https://raider.io/mythic-plus-runs/season-sl-4...,NaN
76,Tauryy,Shattered-Halls,2022-10-10,09:22:11,Mechagon Workshop,WORK,22,1,162.0,0:26:54,1614169,1920999,Fortified,Raging,Explosive,Shrouded,https://raider.io/mythic-plus-runs/season-sl-4...,NaN
77,Tauryy,Shattered-Halls,2022-10-10,08:40:18,Tazavesh: So'leah's Gambit,GMBT,22,1,162.4,0:24:13,1453283,1800999,Fortified,Raging,Explosive,Shrouded,https://raider.io/mythic-plus-runs/season-sl-4...,NaN
